In [1]:
import tensorflow as tf
import skimage.io
import skimage.color
import numpy as np
from pprint import pprint

In [2]:
import read_cifar10 as cf10

#@read_data.restartable
def cifar10_dataset_generator(dataset_name, batch_size, restrict_size=1000):
    assert dataset_name in ['train', 'test']
    assert batch_size > 0 or batch_size == -1  # -1 for entire dataset
    
    X_all_unrestricted, y_all = (cf10.load_training_data() if dataset_name == 'train'
                                 else cf10.load_test_data())
    
    actual_restrict_size = restrict_size if dataset_name == 'train' else int(1e10)
    X_all = X_all_unrestricted[:actual_restrict_size]
    data_len = X_all.shape[0]
    batch_size = batch_size if batch_size > 0 else data_len
    
    X_all_padded = np.concatenate([X_all, X_all[:batch_size]], axis=0)
    y_all_padded = np.concatenate([y_all, y_all[:batch_size]], axis=0)
    
    for slice_i in range(math.ceil(data_len / batch_size)):
        idx = slice_i * batch_size
        #X_batch = X_all_padded[idx:idx + batch_size]
        X_batch = X_all_padded[idx:idx + batch_size]*255  # bugfix: thanks Zezhou Sun!
        y_batch = np.ravel(y_all_padded[idx:idx + batch_size])
        yield X_batch.astype(np.uint8), y_batch.astype(np.uint8)

cifar10_dataset_generators = {
    'train': cifar10_dataset_generator('train', 1000),
    'test': cifar10_dataset_generator('test', -1)
}


In [3]:
#Load cifar-10 data
cf10_tr=cf10.load_training_data()
cf10_tr_img=cf10_tr[0]
cf10_tr_label = cf10_tr[1]

cf10_test=cf10.load_test_data()
cf10_test_img=cf10_test[0]
cf10_test_label = cf10_test[1]

In [4]:
import skimage.io
def img2block(im):
    #lion[:2816,:4224,:]
    #skimage.io.imshow(lion)
    #skimage.io.show()
    
    im = im.astype(np.float32)
    row,col,color = im.shape
    im_bl=np.zeros((int(row*col/1024),32,32,3)).astype(np.float32)
    count=0
    for i in range(0,row-row%32,32):
        for j in range(0,col-col%32,32):
            im_bl[count,:,:,:]=im[i:i+32,j:j+32,:]
            count = count +1
    im_bl=im_bl/255.
    return im_bl

def block2img(img_blocks,img_size):
    
    row,col = img_size
    img=np.zeros((row,col,3)).astype(np.float32)
    n,k,l,c=img_blocks.shape
                 
    for i in range(0,int(row/k)):
        for j in range(0,int(col/k)):
            img[i*k:(i+1)*k,j*l:(j+1)*l,:]=img_blocks[int(i*col/k+j),:,:,:]
    return img

#Get the patches of lena image
lena_img = skimage.io.imread('../test_img/lena512color.tiff')
lena_32=img2block(lena_img)

        

In [5]:
#Function for converting a double to uint8
def convert2uint8(img):
    img[img>255]=255
    img[img<0]=0
    return img.astype(np.uint8)

# Part-2 - CNN

In [6]:
#Create the inputs in the desired format
x_tr = cf10_tr_img.astype(np.float32)#*255.
x_test = cf10_test_img.astype(np.float32)#*255.
x_test=x_test[:200,:,:,:]
img = skimage.io.imread('../test_img/lena512color.tiff')
img_32=img2block(img)


In [7]:
def cnn_autoencoder(x_,kernels1=[5,7],kernels2=[7,5],filters1=[16,128],filters2=[128,3],pool_size=[1,2,2,1],name='autoencoder'):
    '''
    Autoencoder network
    
    Inputs:
    x_ (tf.placeholder) : Input tensor
    kernels1 (1D array) : Size of the encoder kernels (assumed square kernels)
    kernels2 (1D array) : Size of the decoder kernels (assumed square kernels)
    filters1 (1D array) : Number of filters in encoder layers
    filters2 (1D array) : Number of filters in decoder layers
    pool_size (1D array): Pooling size in each layer. Its length must be equal to len(kernels1)+len(kernels2)
                          First len(kernels1) terms will be used as pooling layers of encoder/
                          Remainin terms will be used as unpooling layers of decoder
                          
    Returns:
    out_ (tf.placeholder)     : Output of the autoencoder without quantization in the middle
    out_quant (tf.placeholder): Output of the autoencoder with quantization in the middle
    '''
    with tf.variable_scope(name):
        out_=x_
        for k in range(len(kernels1)):
            conv = tf.layers.conv2d(inputs=out_,
                                    filters=filters1[k],
                                    kernel_size=[kernels1[k],kernels1[k]],
                                    padding="same",
                                    activation=tf.nn.relu,
                                    name='conv'+str(k))
            pool_now=pool_size[k]
            if(pool_now==1):
                out_=conv
            else:
                out_ = tf.layers.max_pooling2d(inputs=conv, 
                                               pool_size=[pool_now,pool_now], 
                                               strides=pool_now,
                                               name = 'pool'+str(k))

            out_quant=tf.round(out_*255.)/255.

        for k in range(len(kernels2)):
            with tf.variable_scope("deconv") as var_scope:
                pool_now=pool_size[k+len(kernels1)]
                if(pool_now==1):
                    x_up=out_
                    out_ = tf.layers.conv2d(inputs=x_up,
                                            filters=filters2[k],
                                            kernel_size=[kernels2[k],kernels2[k]],
                                            padding="same",
                                            activation=tf.nn.relu,
                                            name='deconv'+str(k))
                    var_scope.reuse_variables() 
                    x_quant_up=out_quant
                    out_quant = tf.layers.conv2d(inputs=x_quant_up,
                                                filters=filters2[k],
                                                kernel_size=[kernels2[k],kernels2[k]],
                                                padding="same",
                                                activation=tf.nn.relu,
                                                name='deconv'+str(k))
                else:
                    sh = out_.get_shape().as_list()
                    x_up=tf.image.resize_images(out_,[sh[1]*pool_now,sh[2]*pool_now])
                    out_ = tf.layers.conv2d(inputs=x_up,
                                            filters=filters2[k],
                                            kernel_size=[kernels2[k],kernels2[k]],
                                            padding="same",
                                            activation=tf.nn.relu,
                                            name='deconv'+str(k))
                    var_scope.reuse_variables() 
                    x_quant_up=tf.image.resize_images(out_quant,[sh[1]*pool_now,sh[2]*pool_now])
                    out_quant = tf.layers.conv2d(inputs=x_quant_up,
                                                filters=filters2[k],
                                                kernel_size=[kernels2[k],kernels2[k]],
                                                padding="same",
                                                activation=tf.nn.relu,
                                                name='deconv'+str(k))


    return out_,out_quant


In [8]:
#Working
def apply_classification_loss_mse_with_rnn(kernels1=[5,7],kernels2=[7,5],
                                     filters1=[16,128],filters2=[128,3],
                                    pool_size=[2,2],learning_rate=1.,FT=False,depth =3):
    '''
    MSE based autoencoder optimizer.
    
    Inputs:
    kernels1 (1D array) : Size of the encoder kernels (assumed square kernels)
    kernels2 (1D array) : Size of the decoder kernels (assumed square kernels)
    filters1 (1D array) : Number of filters in encoder layers
    filters2 (1D array) : Number of filters in decoder layers
    pool_size (1D array): Pooling size in each layer. Its length must be equal to len(kernels1)+len(kernels2)
                          First len(kernels1) terms will be used as pooling layers of encoder/
                          Remainin terms will be used as unpooling layers of decoder
    learning_rate(float): Learning rate of the optimizer
    FT (boolean)        : Boolean value for fine-tuning operation on decoder weights
    depth(integer)      
    
    Returns:
    model_dict          : Dictionary of the required output files
    '''
    with tf.Graph().as_default() as g:
        with tf.device("/gpu:0"):  # use gpu:0 if on GPU
            x_ = tf.placeholder(tf.float32, [None, 32, 32, 3])
            
            
            (x_out2,x_out_quant)=cnn_autoencoder(x_,pool_size=pool_size,kernels1=kernels1,filters1=filters1,
                                kernels2=kernels2,filters2=filters2,name='filter0')
            x_out1=x_
            mse_loss1=tf.reduce_mean(tf.subtract(x_out1,x_out2)**2)
            for k in range(1,depth):
                (x_out3,x_out_quant)=cnn_autoencoder(x_out1-x_out2,pool_size=pool_size,kernels1=kernels1,filters1=filters1,
                                kernels2=kernels2,filters2=filters2,name='filter'+str(k))
                mse_loss1=tf.add(mse_loss1,tf.reduce_mean(tf.subtract(x_out1,tf.add(x_out2,x_out3))**2))
                x_out1=x_out2
                x_out2=x_out3
            x_out3=x_out2
            #y_dict = dict(labels=y_, logits=y_logits)
            #losses = tf.nn.sparse_softmax_cross_entropy_with_logits(**y_dict)
            #cross_entropy_loss = tf.reduce_mean(losses)
            #mse_loss1=tf.reduce_mean(tf.subtract(x_,x_out)**2)
            #a=tf.pad(tf.subtract(x_,x_out),[[0,0],[16,16],[16,16],[0,0]],'CONSTANT')

            #mse_loss1=tf.reduce_mean(tf.nn.conv2d(a,h3,strides=[1,1,1,1],padding="VALID")**2)
            mse_loss2=tf.reduce_mean(tf.subtract(x_,x_out3)**2)
            trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            if(FT):
                with tf.variable_scope('deconv', reuse=True) as vs:
                    var_list=[v for v in tf.global_variables() if v.name.startswith(vs.name)]
                train_op = trainer.minimize(mse_loss1,var_list=var_list)
            else:
                train_op = trainer.minimize(mse_loss1)

            #y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            #correct_prediction = tf.equal(tf.cast(y_pred, tf.int32), y_)
            #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    model_dict = {'graph': g, 'inputs': x_,'outputs':x_out_quant, 'train_op': train_op, 'loss1': mse_loss1,'loss2': mse_loss2}
    
    return model_dict

In [9]:
def train_model(model_dict, dataset_generators, train_every=100, test_every=200, load=False,
                learning_rate=1.,fname='cifar10_recon',outname='/tmp/cnn_autoencoder',ftname='/tmp/cnn_autoencoder'):
    '''
    Inputs:
    model_dict: Output of apply_classification_loss_mse
    x_tr      : Training images
    x_test    : Test Images
    x_32      : 32x32 patches of a big image
    load      : Boolean for loading the weights from pre-trained network
    fname     : Directory to save outputs
    outname   : Directory to save (load=False) or load (load=True) weights
    ftname    : Directory to save new weights when load+True
    '''
    with model_dict['graph'].as_default(), tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ### WORK ON IT NOT TOO HARD
        saver=tf.train.Saver()
        if(load):
            saver.restore(sess, outname)
            print("Model loaded")
        else:
            sess.run(tf.global_variables_initializer())
        
        ids=[i for i in range(100)]
        for iter_i in range(10000):
            batch_xs = x_tr[ids,:,:,:] 
            ids=[(ids[0]+100+i)%x_tr.shape[0] for i in range(100)]
            sess.run(model_dict['train_op'], feed_dict={model_dict['inputs']: batch_xs})
            
            # test trained model
            if iter_i % train_every == 0:
                tf_feed_dict = {model_dict['inputs']: batch_xs}
                loss_val = sess.run(model_dict['loss1'], feed_dict={model_dict['inputs']: batch_xs})
                print('iteration %d\t train mse: %.3E\t'%(iter_i,loss_val))
                if iter_i % test_every == 0:
                    #tf_feed_dict = {x_: x_test}
                    loss_val1 = sess.run(model_dict['loss1'], feed_dict={model_dict['inputs']: x_test})
                    loss_val2 = sess.run(model_dict['loss2'], feed_dict={model_dict['inputs']: x_test})
                    print('iteration %d\t TEST MSE: %.3E\t TEST MSE(Quantized): %.3E\t'%(iter_i,loss_val1,loss_val2))
                    
                    
                    lena_block=sess.run(model_dict['outputs'], 
                                       feed_dict={model_dict['inputs']:img_32})
                    x_from_test=sess.run(model_dict['outputs'], 
                                         feed_dict={model_dict['inputs']:x_test[:5,:,:,:].reshape([-1,32,32,3])})
                    
                    lena_recon=block2img(lena_block,(512,512))
                    lena_recon = convert2uint8(lena_recon*255.)
                    skimage.io.imsave('../'+fname+'/lena4_'+str(int(iter_i/test_every))+'.tiff',lena_recon)

                    for i in range(2):
                        img_recon=convert2uint8((255*x_from_test[i,:,:,:]).reshape([32,32,3])).astype(np.uint8)
                        skimage.io.imsave('../'+fname+'/test'+str(i)+'_'+str(int(iter_i/test_every))+'.tiff',img_recon)
                        
        saver = tf.train.Saver()
        if(load):
            outname=ftname
        save_path = saver.save(sess, outname)
        print("Model saved in file: %s" % save_path)
        return saver
                


In [ ]:
## CNN-RNN-AE with best parameters

In [ ]:
#YOU NEED TO CREATE A FOLDER NAMED 'cifar10_recon3' BEFORE RUNNING THAT CODE
tf.reset_default_graph()
model_dict=apply_classification_loss_mse_with_rnn(kernels1=[5,7,9,9],kernels2=[9,7,7,5],
                                         filters1=[128,64,16,4],filters2=[8,8,3,3],
                                         pool_size=[1,2,2,1,1,2,2,1],learning_rate=7e-5)
saver = train_model(model_dict, [], train_every=100, test_every=2000,load=False,
                    fname='cifar10_recon3',outname='/tmp/cnnx4_test2')

iteration 0	 train mse: 4.982E-01	
iteration 0	 TEST MSE: 5.656E-01	 TEST MSE(Quantized): 2.967E-01	


/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon0/lena4_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon0/test0_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mtezcan/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ../cifar10_recon0/test1_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)


iteration 100	 train mse: 2.486E-01	
iteration 200	 train mse: 1.516E-01	
